In [1]:
import os
import pandas as pd
import geopandas as gpd
import folium
import itertools
from pathlib import Path

# Completar con el directorio donde se encuentra clonado e instalado el repositorio de UrbanTrips
URBANTRIPS_PATH = "RUTA/DE/URBANTRIPS"
URBANTRIPS_PATH = Path(URBANTRIPS_PATH)
os.chdir(URBANTRIPS_PATH)

from urbantrips.utils import utils
from urbantrips.kpi import overlapping as ovl
from urbantrips.viz import overlapping as ovl_viz


# Leer archivos de configuración y conexiones a las db
configs = utils.leer_configs_generales()
alias = configs['alias_db_data']
conn_data = utils.iniciar_conexion_db(tipo='data')
conn_insumos = utils.iniciar_conexion_db(tipo='insumos')

In [ ]:
# Se leen los datos de las lineas
metadata_lineas = pd.read_sql("select id_linea,nombre_linea, modo from metadata_lineas;", conn_insumos)
# Se puede buscar por nombre de linea que contenga alguna palabra o numero
metadata_lineas[metadata_lineas.nombre_linea.str.contains("17") #reemplazar 17 por lo que se desee buscar en el nombre de la linea
                ]

In [3]:
# Configurar el día a procesar 
day = "weekday"

# La resolucion h3 (no puede ser mayor a la que aparece en las configuraciones)
h3_res_comp = 8

# Los id de las lineas a comparar
comp_line_id = 1
base_line_id = 1

In [4]:
comparison_info = ovl.get_route_combinations(base_line_id, comp_line_id)

# Aca se pueden ver todos los id de ruta (lineas o ramales) de las lineas a comparar
route_id_combinations = comparison_info['route_id_combinations']
# Aca la metadata de las rutas  (lineas o ramales)
route_metadata= comparison_info['metadata']
route_type = comparison_info['route_type']


In [ ]:
# Selecciona un par de las posibles combinaciones de ramales
route_id_combination = route_id_combinations[0] 
route_id_combination

In [ ]:
# get routes from the combination
base_route_id, comp_route_id = ovl.get_route_ids_from_combination(base_line_id, comp_line_id, route_id_combination)


In [ ]:
# Calcula la superposicion de la oferta de la linea base con la de la linea de comparacion
overlapping_dict = ovl.compute_supply_overlapping(day, base_route_id,comp_route_id,route_type,h3_res_comp)  
base_gdf = overlapping_dict["base"]["h3"]
base_route_gdf = overlapping_dict["base"]["line"]
comp_gdf = overlapping_dict["comp"]["h3"]
comp_route_gdf = overlapping_dict["comp"]["line"]


In [ ]:
# Se visualiza la superposicion de la oferta de la linea base con la de la linea de comparacion
f = ovl_viz.plot_interactive_supply_overlapping(overlapping_dict)
f.save(f"resultados/html/{alias}_supply_overlapping_base_{base_route_id}_comp_{comp_route_id}_h3_{h3_res_comp}.html")
f

In [ ]:
# Calcula la demanda de la linea base y la de comparacion
base_demand,comp_demand = ovl.compute_demand_overlapping(base_line_id,comp_line_id,day,
                                                         base_route_id,comp_route_id,
                                                         base_gdf,comp_gdf)

In [ ]:
# Se visualiza la superposicion de la demanda de la linea base con la de la linea de comparacion
fig = ovl_viz.plot_interactive_demand_overlapping(base_demand, comp_demand, overlapping_dict)
fig.save(f"resultados/html/{alias}_demand_overlapping_base_{base_route_id}_comp_{comp_route_id}_h3_{h3_res_comp}.html")
fig